In [ ]:
'''
GeneCNN, a convolutional neural network-based gene predictor
Developed by Michael Morikone
Requires a genome file and transcriptomic data
Tested with the masked flattened buffalograss genome and 6 transcriptomic datasets from 4 different BioProjects
'''

In [ ]:
import numpy as np
import tensorflow as tf
from Bio import SeqIO
from sklearn.model_selection import train_test_split
import random
import keras_tuner as kt
from tensorflow import keras

In [ ]:
genome = SeqIO.parse("flattened_genome_model_mask.fasta", "fasta") #single sequence masked genome used for training dataset

seqs = []
seqs_ids = []

#converts SeqIO object to list
for sequences in genome:
    seqs.append(str(sequences.seq))
    seqs_ids.append(sequences.id)

#flatten genome and create associated vector with scaffold assignment per base
flattened_genome = []
genome_id = [] 

for i in range(len(seqs_ids)):
    for j in range(len(seqs[i])):
        flattened_genome.append(seqs[i][j])
        genome_id.append(seqs_ids[i])

In [ ]:
#Mapped all transcriptomic data to flattened cleaned genome.
#Created sorted bam files and used samtools depth -aa to generate mapping depth per base and put into a file with depth per mapping file as columns.
#Iterate through lines and iterate through columns. Add all depths to a list, if sum of list is 0, add 0 to true value list. If sum is >0, add 1 to true value list.

true_value = []
with open("flattened_genome_model_mask_samtools_depth.txt") as f:
    for line in f:
        result = []
        result.append(line.split('\t')[2:]) #take only depth
        result[0][-1] = result[0][-1].strip() #remove new line
        result[0] = [int(i) for i in result[0]] #convert list to int
        if sum(result[0])>0:
            true_value.append(1)
        else:
            true_value.append(0)

In [ ]:
#Creation of label lists for dataset

#single sample length
no_length = 0 
yes_length = 0

#single sample sequence
no_current = []
yes_current = []

#all samples
no_list = []
yes_list = []
sequence_length = 500 #highest performance tested from 200-800 bp

for x in range(len(true_value)):
    if no_length < sequence_length and yes_length < sequence_length:
        if true_value[x] == 0:
            no_length += 1
            no_current.append(flattened_genome[x])
            yes_length = 0
            yes_current = []
        elif true_value[x] == 1:
            yes_length += 1
            yes_current.append(flattened_genome[x])
            no_length = 0
            no_current = []
    else:
        if no_length == sequence_length:
            no_list.append(no_current)
            no_current = []
            no_length = 0
        elif yes_length == sequence_length:
            yes_list.append(yes_current)
            yes_current = []
            yes_length = 0

In [ ]:
# oversampling, better performance than undersampling

complement = len(no_list) - len(yes_list)
yes_sample = []
yes_sample.extend(yes_list)
for i in range(complement):
    yes_sample.extend(random.choices(yes_list, k=1)) #keeps original dataset then adds nearly 4x randomly selected data of yes_list to create positive labeled dataset of same size as negative

In [ ]:
#creation of flattened labels to match flatened dataset, in order of all positive samples then all negative samples in one list

#samples
merged_list = []
merged_list.extend(yes_sample) 
merged_list.extend(no_list) 

#create labels
yes_labels = [1] * len(yes_sample)
no_labels = [0] * len(no_list) 

#combine and keep same order as samples
labels = []
labels.extend(yes_labels)
labels.extend(no_labels)

In [ ]:
# data preparation for one hot encoding, conversion of alphabetic representation of nucleotides to numeric representation

ordinal_genome = [] 

for seq in range(len(merged_list)):
    working_seq = []
    for nucleotide in range(len(merged_list[seq])):
        if merged_list[seq][nucleotide] == 'A':
            working_seq.append(0)
        elif merged_list[seq][nucleotide] == 'C':
            working_seq.append(1)
        elif merged_list[seq][nucleotide] == 'G':
            working_seq.append(2)
        else:
            working_seq.append(3)
    ordinal_genome.append(working_seq)

In [ ]:
#data splitting

train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
x_train, x_test, y_train, y_test = train_test_split(ordinal_genome, labels, test_size=1 - train_ratio)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio))

In [ ]:
# one hot encoding for data and labels
genome_category_count = 4
binary_train= tf.one_hot(x_train, genome_category_count)
binary_val= tf.one_hot(x_val, genome_category_count)
binary_test= tf.one_hot(x_test, genome_category_count)

# Convert both true values to Tensors
label_category_count = 2

y_train = tf.one_hot(y_train, label_category_count)
y_val = tf.one_hot(y_val, label_category_count)
y_test = tf.one_hot(y_test, label_category_count)

In [ ]:
#confirm shaping is correct

print(binary_train.shape)
print(tf.convert_to_tensor(y_train).shape)
print(binary_train[0].shape)

In [ ]:
#model definition for manual tweaking, uncomment if needed and comment out next three code blocks instead

'''#Modeling
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Flatten, BatchNormalization
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Conv1D(filters=10, kernel_size=3, input_shape=(binary_train.shape[1], 4), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=4))
model.add(Conv1D(filters=20, kernel_size=3, activation='relu')) 
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=4))
model.add(Conv1D(filters=40, kernel_size=5, activation='relu')) 
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=4))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=['binary_accuracy'])
model.summary()'''

In [ ]:
#model definition for tuning

def build_model(hp):
    model = keras.Sequential([
    keras.layers.Conv1D(filters=hp.Int('conv1_filter', min_value=4, max_value=128, step=4), kernel_size=hp.Choice('conv1_kernel', values=[3,5]), activation='relu', input_shape=(binary_train.shape[1], 4)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling1D(pool_size=4),
    keras.layers.Conv1D(filters=hp.Int('conv2_filter', min_value=4, max_value=128, step=4), kernel_size=hp.Choice('conv2_kernel', values=[3,5]), activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling1D(pool_size=4),
    keras.layers.Conv1D(filters=hp.Int('conv3_filter', min_value=4, max_value=128, step=4), kernel_size=hp.Choice('conv3_kernel', values=[3,5]), activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling1D(pool_size=4),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(2, activation='softmax')])
    
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3])), loss='binary_crossentropy', metrics=['binary_accuracy'])
    return model

In [ ]:
#BayesianOptimization tuning

from keras_tuner import BayesianOptimization

tuner = BayesianOptimization(build_model, objective='val_binary_accuracy', max_trials=15, project_name='Bayesian2')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
tuner.search(binary_train, y_train, epochs=20, validation_data=(binary_val,y_val), callbacks=[stop_early])

In [ ]:
#model layout

model=tuner.get_best_models(num_models=1)[0]
model.summary()

In [ ]:
#model training and loss plot

history = model.fit(binary_train, y_train, epochs=20, validation_data=(binary_val,y_val))

import matplotlib.pyplot as plt

plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
#Accuracy plot

plt.figure()
plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
#test loss and accuracy
results = model.evaluate(binary_test, y_test)
print("test loss, test acc:", results)

In [ ]:
#Confusion matrix
from sklearn.metrics import confusion_matrix
import itertools

predicted_labels = model.predict(np.stack(binary_test))
cm = confusion_matrix(np.argmax(y_test, axis=1),
                      np.argmax(predicted_labels, axis=1))
print('Confusion matrix:\n',cm)

cm = cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]

plt.imshow(cm, cmap=plt.cm.Blues)
plt.title('Normalized confusion matrix')
plt.colorbar()
plt.xlabel('True label')
plt.ylabel('Predicted label')
plt.xticks([0, 1]); plt.yticks([0, 1])
plt.grid('off')
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], '.2f'),
             horizontalalignment='center',
             color='white' if cm[i, j] > 0.5 else 'black')

In [ ]:
#F1 score
loss, f1 = model.evaluate(binary_test, y_test, verbose=0)
print('F1 Score: %.3f' % f1)

In [ ]:
#save model
model.save("Bayesian2")

In [ ]:
#use to visualize model layout as a figure

from keras.utils.vis_utils import plot_model
import pydot, graphviz
plot_model(model, to_file='model_plot_noNone.png', show_shapes=True, show_layer_names=True)